# Seminar Project: Subsetting Speeches
---------------
``` 

Jakob Amann
Elias Gabriel Heppner, 1244414
Andri Rutschmann

### Loading Packages


In [1]:
import numpy as np 
import pandas as pd 
import time
import pickle
import time
import os
import datetime
import zipfile
from tqdm.auto import tqdm


c:\Users\elias\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Speeches Data


In [2]:
#Define 
os.chdir("../data/opendiscourse")


FileNotFoundError: [Errno 2] No such file or directory: 'sub_speches.zip'

In [5]:

zip_file_path = "sub_speeches.zip"

# Name of the CSV file inside the zip file
csv_filename = "sub_speeches.csv"

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Check if the CSV file exists in the zip file
    if csv_filename in zip_ref.namelist():
        # Extract the CSV file
        with zip_ref.open(csv_filename) as csv_file:
            # Read the CSV file using pandas
            df = pd.read_csv(csv_file)
            print(df.head())  # Display the first few rows of the dataframe
    else:
        print(f"{csv_filename} not found in the zip file.")


     id_x  session  electoral_term        date  \
0  810543       79              18  2015-01-15   
1  812133       86              18  2015-02-06   
2  812135       86              18  2015-02-06   
3  812137       86              18  2015-02-06   
4  812836       89              18  2015-02-27   

                                        document_url  \
0  https://dip21.bundestag.de/dip21/btp/18/18079.pdf   
1  https://dip21.bundestag.de/dip21/btp/18/18086.pdf   
2  https://dip21.bundestag.de/dip21/btp/18/18086.pdf   
3  https://dip21.bundestag.de/dip21/btp/18/18086.pdf   
4  https://dip21.bundestag.de/dip21/btp/18/18089.pdf   

                                      speech_content  politician_id  \
0  Frau Präsidentin! Meine Damen und Herren! Lieb...       11003801   
1  Frau Präsidentin! Meine lieben Kolleginnen und...       11003801   
2                                     Ja, bitte.\n\n       11003801   
3  Es ist gar keine Frage, dass es hier zu Übersc...       11003801   
4  Viel

# Preprocessing

In [ ]:
stopword removal, subsampling

# Groundtruthing

## Manifestoberta

In [6]:
# https://huggingface.co/manifesto-project/manifestoberta-xlm-roberta-56policy-topics-sentence-2023-1-1

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("manifesto-project/manifestoberta-xlm-roberta-56policy-topics-sentence-2023-1-1")
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")

sentence = "We will restore funding to the Global Environment Facility and the Intergovernmental Panel on Climate Change, to support critical climate science research around the world"

inputs = tokenizer(sentence,
                   return_tensors="pt",
                   max_length=200,  #we limited the input to 200 tokens during finetuning
                   padding="max_length",
                   truncation=True
                   )

logits = model(**inputs).logits

probabilities = torch.softmax(logits, dim=1).tolist()[0]
probabilities = {model.config.id2label[index]: round(probability * 100, 2) for index, probability in enumerate(probabilities)}
probabilities = dict(sorted(probabilities.items(), key=lambda item: item[1], reverse=True))
print(probabilities)
# {'501 - Environmental Protection: Positive': 67.28, '411 - Technology and Infrastructure': 15.19, '107 - Internationalism: Positive': 13.63, '416 - Anti-Growth Economy: Positive': 2.02...

predicted_class = model.config.id2label[logits.argmax().item()]
print(predicted_class)
# 501 - Environmental Protection: Positive


config.json: 100%|██████████| 5.59k/5.59k [00:00<?, ?B/s]
c:\Users\elias\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\elias\.cache\huggingface\hub\models--manifesto-project--manifestoberta-xlm-roberta-56policy-topics-sentence-2023-1-1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-developmen

ConnectionError: (MaxRetryError("HTTPSConnectionPool(host='cdn-lfs.huggingface.co', port=443): Max retries exceeded with url: /repos/d4/1b/d41b321fd2eba2a7e782aed55a297d1f04a1d962e75d65b8dd9b4499a41af700/fc2f1ab6cbe70afa622f19e2feb559ca04214a52fe0181a8367726931126b00d?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1709907719&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwOTkwNzcxOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9kNC8xYi9kNDFiMzIxZmQyZWJhMmE3ZTc4MmFlZDU1YTI5N2QxZjA0YTFkOTYyZTc1ZDY1YjhkZDliNDQ5OWE0MWFmNzAwL2ZjMmYxYWI2Y2JlNzBhZmE2MjJmMTllMmZlYjU1OWNhMDQyMTRhNTJmZTAxODFhODM2NzcyNjkzMTEyNmIwMGQ~cmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=Mp3OQuTfMG7DIYJ6t6TGADW9NigC2zMCDt7k~iOjYhikAbRWrMOZl6rjvmUPGtUZms7Yy-jZANO0cqiwD-gsqk9jjF0rnvhg